In [14]:
import cv2 as cv
import pytesseract
pytesseract.pytesseract.tessaract_cmd = r'/usr/local/Cellar/tesseract/4.1.1/bin/tesseract'
import nltk
import os 
import re
import matplotlib.pyplot as plt
from skimage import data
from skimage.color import rgb2gray
from PIL import Image
import PIL.ImageOps 
os.chdir('/Users/jpjansen/Desktop/3_28_21/images')
import numpy as np
import glob
import xlsxwriter
from pytesseract import Output

In [13]:
print(os.getcwd())


/Users/jpjansen/Desktop/3_28_21/images


In [3]:
#save as grayscale
for filename in os.listdir(os.getcwd()):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        if filename == '.DS_Store':
            continue
        #image = Image.open(filename)
        #inverted_image = ImageOps.invert(image)
        img = Image.open(filename).convert('LA')
        export = filename.split(".")
        img.save(export[0]+'gray.png')

In [ ]:

image_one = 0
image_two = 0 
image_three = 0
image_four = 0
image_one_data = [] 
image_three_data = [] 
inflows = []
outflows = [] 
image_two_data = []
image_threes = [] 
image_four_data = [] 
stock=['FUBO', 'TME', 'WAFU', 'NEXT', 'XSPA', 'CIDM', 'EEIQ', 'NAKD', 'CLEU', 'GBR', 'HSDT', 'BB','DS', 'KDMN', 'OXY', 'GNPK', 'HOL', 'NPA', 'NSH', 'SFTW', 'SRAC', 'WIMI', 'LGHL', 'NAT', 'SIEN', 'HGEN', 'OXBR', 'DFFN', 'EVFM', 'HEPA', 'CLBS', 'HUGE', 'ENVB', 'DSS', 'EBON', 'CSCW', 'RAIL', 'ENSV', 'HUSA', 'AYRO', 'VISL', 'BRQS', 'BYFC', 'BOWX', 'DLPN', 'HOFV', 'JFIN', 'LMPX', 'UAVS', 'AMC', 'LIFE', 'ATNF', 'AACG', 'ORN', 'PLUG', 'CASI', 'ADA', 'ZDGE', 'FLNT', 'CURI', 'MX', 'TH', 'GHSI', 'KNDI', 'IZEA', 'MTNB', 'OEG', 'PRQR', 'TKAT', 'RCON', 'AJAX'] 
free_float = [] 
picture_detection = [] 
noise_list = [',','.',r'/', '(', ')']
market_names = ['NASDAQ','NASDA','NASD', 'NAS', 'NA','NYSE', 'NYS','NY','NYSEARCA' ,'AMEX', 'AME', 'AM']
undetermined_names = []

for filename in sorted(os.listdir(os.getcwd())):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        if filename == '.DS_Store':
            continue
        print(filename) 
        img = cv.imread(filename)
        #img =  rgb2gray(img)
        ret,thresh1 = cv.threshold(img,241,255,cv.THRESH_BINARY)
        #kernel = np.ones((5,5), np.uint8)
        #img_dilation = cv.dilate(thresh1, kernel, iterations=10) 
        #plt.imshow(img)
        #ret,thresh1 = cv2.threshold(img,0,255,cv2.THRESH_OTSU)
        #plt.imshow(img)
        
        #--psm 12 -c tessedit_char_whitelist= 0123456789
        text = pytesseract.image_to_string(thresh1,config='--psm 6')
        #print(text)
        nltk_tokens = nltk.word_tokenize(text)
        get_stock_name=[]
        for i in range(len(nltk_tokens)):
            if nltk_tokens[i] not in noise_list:
                get_stock_name.append(nltk_tokens[i])
        print(nltk_tokens)
        #cv.imshow('picture',thresh1)
        #cv.waitKey(0)
        print('FILENAME',filename)
        print(get_stock_name)
        #print(nltk_tokens)
        for i in range(len(get_stock_name)):
            if get_stock_name[i] in market_names:
                if get_stock_name[i-1] in stock:
                    name = get_stock_name[i-1]
                    print(name)
                break
            else:
                name = 'NaN'
                print('Couldnt determine name')
        if name=='NaN':
            undetermined_names.append(filename)
                       
        for i in range(len(nltk_tokens)):
            multiplier = 1 
            #if nltk_tokens[i] in stock:
                #name = nltk_tokens[i]
            if nltk_tokens[i]=="NEXT" and nltk_tokens[i+1]=="EARNINGS":
                print("image 4 detected")
                #print(nltk_tokens)
            if nltk_tokens[i]=="EX-DIV" and nltk_tokens[i+1]=='DATE':
                free_float_value = nltk_tokens[i+2]
                if free_float_value=="--":
                    free_float.append((name,'NaN'))
                    break
                print(free_float_value)
                if 'K' in free_float_value:
                    free_float_value = free_float_value.replace("K","")
                    multiplier = .001
                elif 'M' in free_float_value:
                    multiplier = 1
                    free_float_value = free_float_value.replace("M","")
                elif 'B' in free_float_value:
                    multiplier = 1000
                    free_float_value = free_float_value.replace("B","")
                elif 'T' in free_float_value:
                    multiplier = 1000000
                    free_float_value = free_float_value.replace("T","")
                if "." not in free_float_value and "," in free_float_value:
                    commas = []
                    for i in range(len(free_float_value)):
                        if free_float_value[i]==",":
                            commas.append(i)
                    replace = np.max(commas)
                    new_str = list(free_float_value)
                    del(new_str[replace])
                    new_str.insert(replace,".")
                    final = ''.join(new_str)
                    final.replace(",","")
                    free_float_value = final
                elif "." and "," in free_float_value:
                    continue
                if free_float_value != 'NaN':
                    free_float.append((name,float(free_float_value)*multiplier))
                multiplier = 1 
                
                picture_detection.append((name,filename,4))       
                image_four+=1
                #for i in range(len(nltk_tokens)):
                    #if nltk_tokens[i] == 
                
                image_four_data.append(nltk_tokens)
                picture_detection.append((name,filename,4))
                break
            if nltk_tokens[i]=="Order" and nltk_tokens[i+1]=="Flow" and nltk_tokens[i+2]=="Distribution":
                print("Image 2 detectd")
                multiplier = 1
                inflow = 'NaN'
                outflow = 'NaN'
                picture_detection.append((name,filename,2))
                for i in range(len(nltk_tokens)):
                    if nltk_tokens[i]=='Inflow':
                        inflow = nltk_tokens[i+2].replace(",","")
                        print('Inflow',inflow)
                    if nltk_tokens[i]=='Outflow':
                        outflow = nltk_tokens[i+2].replace(",","")
                        print('Outflow', outflow)
                for i in range(len(nltk_tokens)):
                    if nltk_tokens[i]=='Million' and nltk_tokens[i+1] =='USD':
                        multiplier = 1000000
                        break
                    if nltk_tokens[i]=='Kilo' and nltk_tokens[i+1]=='USD':
                        multiplier = 1000 
                        break
                image_two+=1
                inflows.append((name,float(inflow)*multiplier))
                outflows.append(float(outflow)*multiplier)
                #image_two_data.append((name,float(inflow*multiplier),float(outflow*multiplier)))
                multiplier = 1 
                break 
            if nltk_tokens[i]=="Large" and nltk_tokens[i+1]=="Scale" and nltk_tokens[i+2]=="Orders" and nltk_tokens[i+3] == "in" and nltk_tokens[i+4] == "Last"and nltk_tokens[i+5] == "5"and nltk_tokens[i+6] == "Days":
                print("image 3 detected")
                picture_detection.append((name,filename,3))
                img_3_export = filename.split(".")
                exp_image3 = Image.open(filename)
                export = filename.split(".")
                exp_image3.save('/Users/jpjansen/Desktop/3_28_21/image threes/'+export[0]+'image3.png')
                #print(nltk_tokens)
                multiplier = 1 
                if nltk_tokens[i+7]=="Kilo":
                    large_multiplier = 1000
                if nltk_tokens[i+7]=="Million":
                    large_multiplier = 1000000
                if nltk_tokens[i+7]=="USD": 
                    large_multiplier = 1
                image_threes.append((name,filename,large_multiplier))
                image_three+=1
                break
          
            if nltk_tokens[i]=='Order' and nltk_tokens[i+1]=='Book' and nltk_tokens[i+2]=='-'and nltk_tokens[i+3]=='Level':
                # old img one detection"AMA" and "DMA" in nltk_tokens:
                print("image 1 detected")
                #print(nltk_tokens)
                image_one+=1
                #image_one_data.append(nltk_tokens)
                #for i in range(len(nltk_tokens)):
                    #if nltk_tokens[i]=='NASDAQ'or nltk_tokens[i]=='NYSE'or nltk_tokens[i]=='AMEX':
                        #name = nltk_tokens[i-1]
                        #break
                #img = cv.imread(filename)
                #ret,thresh1 = cv.threshold(img,235,255,cv.THRESH_BINARY)
                #kernel = np.ones((5,5), np.uint8)
                #img_dilation = cv.dilate(thresh1, kernel, iterations=10)
                #cv.imshow('picture',thresh1)
                #cv.waitKey(0)
                #text = pytesseract.image_to_string(thresh1,config='--psm 6')
                #nltk_tokens = nltk.word_tokenize(text)
                for i in range(len(nltk_tokens)):
                    if re.match(r"MA10:",nltk_tokens[i])!=None:
                        ten_day_ma = nltk_tokens[i]
                        if "." not in ten_day_ma:
                            try:
                                commas = []
                                for i in range(len(ten_day_ma)):
                                    if ten_day_ma[i]==",":
                                        commas.append(i)
                                replace = np.max(commas)
                                new_str = list(ten_day_ma)
                                del(new_str[replace])
                                new_str.insert(replace,".")
                                final = ''.join(new_str)
                                final.replace(",","")
                                ten_day_ma = float(final[1])
                                
                                break
                            except:
                                print('VALUE ERROR')
                                ten_day_ma = 'NaN'
                                break
                        else:
                            ten_day_ma.replace(",","")
                            ten_day_ma = ten_day_ma.split(":")
                            ten_day_ma = float(ten_day_ma[1])
                            break
                print(ten_day_ma)
                #ten_day_ma = ten_day_ma.split(":")
                #print(ten_day_ma)
                picture_detection.append((name,filename,1))
                image_one_data.append((name,ten_day_ma))
                break
            
print(image_one,image_two,image_three,image_four)
#print("Image One data:", image_one_data)
#print("Image Two data:", image_two_data)
#print("Image Three data:", image_three_data)
print("Image Four data:",image_four_data)
print(free_float)
print(inflows)
print(outflows)
print(image_one_data)
print(np.asarray(picture_detection))

IMG_6720gray.jpeg
['1:18', '4', 'all', '>', '<', '4', 'Search', 'F', '<', 'UBO', 'NYSE', 'vy', 'fuboTV', 'Inc.', '-4.01', '21', '.88', '15.49', '%', 'eB', '-8.70', '(', '-27.69', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '26.00/25.89', '20.69-26.67', '31.75M', '1D', '5D', '1M', '3M', '1Y', 'SY', 'Max', '|A', '88', 'MA5:22.70', 'MA10:22.65', 'MA20:22.60', '1.92', '+1.58', '%', '2391s', '.', '!', '.', '7.97', '%', '25.91', 'i', '|', 'nN', '.', '7', ':', '!', '|', 'le', 'ed', 'nm', '03/22', '03/23', '03/24', '03/25', '03/26', '03/27', 'DMA', '(', '10,50,10', ')', 'DDD:0.27', 'AMA:0.32', 'Oe', ',', ')', 'So', ',', 'a', 'NKr7knr', 'fF', 'fi', 'Ny', '.', '\\', '\\S', 'V', '\\P', 'e', 'e', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'Order', 'Book', '-', 'Level', '1', '333', 'Trade', '&', 'e', 'eee', 'Positions', 'Options', 'Comment', 'More', 'ee']
FILENAME IMG_6720gray.jpeg
['1:18', '4', 'all', '>', '<', '4', 'Search', 'F', '<', 'UBO', '

['11:18', '7', 'all', '>', '&', ')', '<', '4', 'Search', 'TME', 'NYSE', '<', '20.10', '-0.26', '-1.28', '%', 'Ww', 'Quotes', 'News', '’', 'Comments', '”', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Million', 'USD', '2.50', '%', '2.16', '%', '27.06', '%', '1', '26.43', '%', 'Orders', '22.35', '%', 'Nw', '533', '521', '441', '384', '49', ';', 'i', '43', ';', 'i', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small', 'Inflow', ':', '1,024', 'Outflow', ':', '947', 'Large', 'Scale', 'Orders', 'in', 'Last', '5', 'Days', 'Million', 'USD', '17', '7', '7', '4', '“', '1', 'Positions', 'Options', 'Comment', 'More', 'ee']
FILENAME IMG_6725gray.jpeg
['11:18', '7', 'all', '>', '&', '<', '4', 'Search', 'TME', 'NYSE', '<', '20.10', '-0.26', '-1.28', '%', 'Ww', 'Quotes', 'News', '’', 'Comments', '”', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Million', 'USD', '2.50', '%', '2.16', '%', '27.06', '%', '1', '26.43', '%', 'Orders', '22.35', '%', 'N

['11:18', '7', 'all', 'F', '<', '4', 'Search', '<', 'WAFU', 'NASDAQ', 'wy', '12.66', '+7.91', '+166.53', '%', '+', 'e', 'eo', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', '10.37', '%', '10.12', '%', '=f', '=', '10.17', '%', '~', '...', '198', '188', '67', ';', '65', '65', 'i', '60', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small', 'Inflow', ':', '330', 'Outflow', ':', '313', 'Large', 'Scale', 'Orders', 'in', 'Last', '5', 'Days', 'Kilo', 'USD', '1,620', '0', '0', '-28', '0', '|', '03/22', '03/23', '03/24', '03/25', '03/26', 'Trade', 'e', 'a', '2', '333', 'Positions', 'Alert', 'Comment', 'More', 'ee']
FILENAME IMG_6730gray.jpeg
['11:18', '7', 'all', 'F', '<', '4', 'Search', '<', 'WAFU', 'NASDAQ', 'wy', '12.66', '+7.91', '+166.53', '%', '+', 'e', 'eo', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', '10.37', '%', '10.12', '%', '=f', '=', '10.17', '%', '~', '...', '198', '188', '67', ';', '65', '65', 'i', '60', 'Large', 'Medium', 'Small', 'Large', 'M

['11:18', '7', 'all', 'F', '&', '<', '4', 'Search', '<', 'NEXT', 'NASDAQ', 'wv', '2.570', '+0.620', '+31.79', '%', '+', 'e', 'e', '.', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'MW', '28.83', '%', '36.14', '%', 'Orders', 'g', '12,39', '%', '15.75', '%', '8,608', '6,867', '3,751', 'i', '2,943', '656', '987', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small', 'Inflow', ':', '13,016', 'Outflow', ':', '10,796', 'Large', 'Scale', 'Orders', 'in', 'Last', '5', 'Days', 'Kilo', 'USD', '74', '33', '-37', '0', '=', '|', '|', '-330', '03/22', '03/23', '03/24', '03/25', '03/26', 'NEXTW', '(', 'OTCMKTS', ')', '0.0280', '0.0000', '0.00', '%', 'Trade', 'e', 'a', '2', '333', 'Positions', 'Alert', 'Comment', 'More', 'ee']
FILENAME IMG_6734gray.jpeg
['11:18', '7', 'all', 'F', '&', '<', '4', 'Search', '<', 'NEXT', 'NASDAQ', 'wv', '2.570', '+0.620', '+31.79', '%', '+', 'e', 'e', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'MW', '28.83', '%', '36.14', '%', 'Order

['11:19', '7', 'all', 'F', '<', '4', 'Search', '<', 'XSPA', 'NASDAQ', 'wv', '1.770', '0.000', '0.00', '%', '+', 'e', 'e', '.', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'Gf', '20.32', '%', '17.38', '%', 'o', 'Orders', '25.39', '%', '10.76', '%', '11.67', '%', '851', '681', ';', '583', '485', 'i', 'i', '361', '390', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small', 'Inflow', ':', '1,429', 'Outflow', ':', '1,923', 'Large', 'Scale', 'Orders', 'in', 'Last', '5', 'Days', 'Kilo', 'USD', '-372', '-471', '-62', '-345', '-196', '03/22', '03/23', '03/24', '03/25', '03/26', 'oes', 'Trade', '&', 'e', 'eee', 'Positions', 'Options', 'Comment', 'More', 'ee']
FILENAME IMG_6738gray.jpeg
['11:19', '7', 'all', 'F', '<', '4', 'Search', '<', 'XSPA', 'NASDAQ', 'wv', '1.770', '0.000', '0.00', '%', '+', 'e', 'e', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'Gf', '20.32', '%', '17.38', '%', 'o', 'Orders', '25.39', '%', '10.76', '%', '11.67', '%', '851', '681', ';',

...

['11:43', '7', 'all', 'SF', '&', ')', 'MX', 'NY', '<', 'oF', 'Ww', 'Magnachip', '+5.60', '26.01', '+27.AA', '%', 'SEB', '+3,63', '(', '+16.15', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '26.55/20.41', '25.25-26.70', '12.22M', '1D', '5D', '1M', '3M', '1Y', 'SY', 'Max', '|A', '88', 'MA5:26.10', 'MA10:26.08', 'MA20:26.11', '26.90', '}', '-', '#', '19.74', '%', '249200', '|', '|', '|', '|', '294', '|', '|', '|', 'Prt', 'nanan', 'nant', 'nn', 'nancy', '-', '|', 'ye', '20.96', '!', '\\', 'A', 'po', 'a', 'Wi', 'ne', '|', 'i', '.', "'", '>', '>', '1', '|', '!', 'Wr', '!', '|', '|', '|', '@', 'a', 'i', 'tn', '1', 'et', 'no', 'nant', '__', 'ce', 'Mette', '03/22', '10:00', '03/24', '03/25', '03/26', '12:00', '03/27', 'DMA', '(', '10,50,10', ')', 'DDD:0.13', 'AMA:0,19', '|', '3.36', '1.47', 'aX', '2', 'Lo', 'oo', 'J', '\\', 'a', '=', 'Wa', 'woe', 'e', 'e', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'Order', 'Book', '-', 'Level', '1', '333', '

['11:43', '7', 'ull', 'S', '&', ')', 'TH', 'NASDAQ', '<', '2.330', '+0.470', '+25', ',', '.27', '%', 'Ww', 'Quotes', 'News', '’', 'Comments', '”', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Kilo', 'USD', '9.64', '%', '1.90', '%', ',', 'i', '17.98', '%', '34.66', '%', '(', 'Orders', '>', '~', 'a', '528', '378', '274', '147', '167', '29', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small', 'Inflow', ':', '1,053', 'Outflow', ':', '469', 'Large', 'Scale', 'Orders', 'in', 'Last', '5', 'Days', 'Kilo', 'USD', '118', 'THWWW', '(', 'NASDAQ', ')', '0.1556', '-0.0101', '-6.10', '%', 'Positions', 'Alert', 'Comment', 'More', 'ee']
FILENAME IMG_6962gray.jpeg
['11:43', '7', 'ull', 'S', '&', 'TH', 'NASDAQ', '<', '2.330', '+0.470', '+25', '.27', '%', 'Ww', 'Quotes', 'News', '’', 'Comments', '”', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Kilo', 'USD', '9.64', '%', '1.90', '%', 'i', '17.98', '%', '34.66', '%', 'Orders', '>', '~', 'a', '528', 

['11:43', '7', 'ull', 'S', '&', ')', 'GHSI', 'NASDAQ', '<', '2.600', '-0.060', '-2.26', '%', 'Ww', 'Quotes', 'News', '’', 'Comments', '”', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Kilo', 'USD', '12.92', '%', 'gu', '23.37', '%', '29.57', '%', 'Orders', '20.48', '%', '13.66', '%', '531', '420', '368', '232', '246', '0', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small', 'Inflow', ':', '1,009', 'Outflow', ':', '788', 'Large', 'Scale', 'Orders', 'in', 'Last', '5', 'Days', 'Kilo', 'USD', '524', '232', '101', '-17', '“', '112', 'Positions', 'Alert', 'Comment', 'More', 'ee']
FILENAME IMG_6966gray.jpeg
['11:43', '7', 'ull', 'S', '&', 'GHSI', 'NASDAQ', '<', '2.600', '-0.060', '-2.26', '%', 'Ww', 'Quotes', 'News', '’', 'Comments', '”', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Kilo', 'USD', '12.92', '%', 'gu', '23.37', '%', '29.57', '%', 'Orders', '20.48', '%', '13.66', '%', '531', '420', '368', '232', '246', '0', 'Large', 'Medium

['11:43', '7', 'ull', 'S', '&', ')', 'KNDI', 'NASDAQ', '<', '5.93', '-0.06', '-1.00', '%', 'Ww', 'Quotes', 'News', '’', 'Comments', '”', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Kilo', 'USD', '17.64', '%', 'gn', '26.40', '%', 'Orders', '24.29', '%', '31.67', '%', '2,229', '1,711', '1,860', '1,243', '0', '0', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small', 'Inflow', ':', '2,954', 'Outflow', ':', '4,089', 'Large', 'Scale', 'Orders', 'in', 'Last', '5', 'Days', 'USD', '0', '0', '0', '0', '0', 'Positions', 'Options', 'Comment', 'More', 'ee']
FILENAME IMG_6970gray.jpeg
['11:43', '7', 'ull', 'S', '&', 'KNDI', 'NASDAQ', '<', '5.93', '-0.06', '-1.00', '%', 'Ww', 'Quotes', 'News', '’', 'Comments', '”', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Kilo', 'USD', '17.64', '%', 'gn', '26.40', '%', 'Orders', '24.29', '%', '31.67', '%', '2,229', '1,711', '1,860', '1,243', '0', '0', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small',

['11:44', '7', 'ull', 'S', '&', ')', 'IZEA', 'NASDAQ', '<', '4.050', '-0.200', '-4.71', '%', 'Ww', 'Quotes', 'News', 'Comments', '”', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Kilo', 'USD', '4.50', '%', '6.99', '%', '26.48', '%', 'T', ':', '29.75', '%', 'Orders', '15.77', '%', '»', '16.51', '%', '2,586', '2,302', '1,371', '1,432', '391', '608', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small', 'Inflow', ':', '4,065', 'Outflow', ':', '4,626', 'Large', 'Scale', 'Orders', 'in', 'Last', '5', 'Days', 'Kilo', 'USD', '134', 'wb', 'Positions', 'Options', 'Comment', 'More', 'ee']
FILENAME IMG_6974gray.jpeg
['11:44', '7', 'ull', 'S', '&', 'IZEA', 'NASDAQ', '<', '4.050', '-0.200', '-4.71', '%', 'Ww', 'Quotes', 'News', 'Comments', '”', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Kilo', 'USD', '4.50', '%', '6.99', '%', '26.48', '%', 'T', ':', '29.75', '%', 'Orders', '15.77', '%', '»', '16.51', '%', '2,586', '2,302', '1,371', '1,432', '

In [8]:
os.chdir('/Users/jpjansen/Desktop/3_28_21/image threes')

large_scale_buying = [] 
counter = 0 
image_threes = np.asarray(image_threes)
image_three_resave = []
for filename in sorted(os.listdir(os.getcwd())):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        if filename == '.DS_Store':
            continue
        print(filename)
        stock_name = image_threes[counter,0]
        final_multiplier = image_threes[counter,2]
        image_three_resave.append(filename)
        counter+=1
        #,cv.IMREAD_GRAYSCALE
        img = cv.imread(filename)
        height = img.shape[0]
        print('height', height)
        ret,thresh1 = cv.threshold(img,240,255,cv.THRESH_BINARY)
        height_2 = thresh1.shape[0]
        print('Height 2', height_2)
        kernel = np.ones((2,2), np.uint8)
        img_dilation = cv.erode(thresh1, kernel, iterations=3) 
        boxes = pytesseract.image_to_data(thresh1,output_type= Output.DICT,config='--psm 6 --oem 3 -c tessedit_char_whitelist=/.,-0123456789')
        good_words = [] 
        good_top = [] 
        remove = ['/',',','.','','-', None]
        for z in range(len(boxes['text'])):
            #print(len(boxes['text'][z]))
            if len(boxes['text'][z])==0:
                #print('Zero')
                continue
            if boxes['text'][z] not in remove:
                good_words.append(boxes['text'][z])
                good_top.append(boxes['top'][z])
        #print('good words', good_words)
        #print('good top', good_top)
        buy_locations = []
        buy_values = []
        for i in range(len(good_words)):
            #print(i, nltk_tokens[i])
                if re.match(r"[0-9][0-9]/[0-9][0-9]",good_words[i])!=None:
                    if re.match(r"[0-9][0-9]/[0-9][0-9]",good_words[i+1])!=None:
                        if good_words[i-6]=='5':
                            print(filename)
                            print(good_words)
                            print(good_top[i]/height)
                            date_dist = good_top[i]/height
                            print('MATCH')
                            final_multiplier = float(final_multiplier)
                            print('buy 1')
                            buy1 = good_words[i-1].replace(",","")
                            print('buy 1',buy1)
                            buy1_loc = (good_top[i-1]/height)-date_dist
                            print('buy 1 location', buy1_loc)
                            buy2_loc = (good_top[i-2]/height)-date_dist
                            buy2 = good_words[i-2].replace(",","")
                            print('buy 2',buy2)
                            print('buy 2 location', buy2_loc)
                            buy3 = good_words[i-3].replace(",","")
                            print('buy 3',buy3)
                            buy3_loc = (good_top[i-3]/height)-date_dist
                            print('buy 3 location', buy3_loc)
                            buy4 = good_words[i-4].replace(",","")
                            print('buy 4',buy4)
                            buy4_loc = (good_top[i-4]/height)-date_dist
                            print('buy 4 location', (good_top[i-4]/height)-date_dist)
                            buy5 = good_words[i-5].replace(",","")
                            print('buy 5',buy5)
                            buy5_loc = (good_top[i-5]/height)-date_dist
                            print('buy 5 locaion', (good_top[i-5]/height)-date_dist)
                            buy_locations.append(buy1_loc)
                            buy_locations.append(buy2_loc)
                            buy_locations.append(buy3_loc)
                            buy_locations.append(buy4_loc)
                            buy_locations.append(buy5_loc)
                            buy_values.append(buy1)
                            buy_values.append(buy2)
                            buy_values.append(buy3)
                            buy_values.append(buy4)
                            buy_values.append(buy5)
                            break
                        else:
                            print('More than 5 values')
                            print(good_words)
                            buy_locations.append('Five val problem')
                            buy_locations.append('Five val problem')
                            buy_locations.append('Five val problem')
                            buy_locations.append('Five val problem')
                            buy_locations.append('Five val problem')
                            buy_values.append('Five val problem')
                            buy_values.append('Five val problem')
                            buy_values.append('Five val problem')
                            buy_values.append('Five val problem')
                            buy_values.append('Five val problem')
                            break
        buy_values_floats = [] 
        for i in range(len(buy_values)):
            try:
                buy_values_floats.append(float(buy_values[i])*final_multiplier)
            except ValueError:
                try:
                    check_7 = buy_values[i].replace("/","7")
                    buy_values_floats.append(float(check_7)*final_multiplier)
                except ValueError:
                    buy_values_floats.append('NaN')
                
        final_values = [] 
        zero_or_negative = 'NaN'
        zero_or_negative_matrix = []
        positive_check = []
        positive_check_loc = [] 
        ##check to see if the location of smallest positive number is higher than the location
        #of a larger positive number (would indicate negative)
        for i in range(len(buy_values_floats)):
            if type(buy_values_floats[i]) == str:
                continue
            if buy_values_floats[i]>0:
                positive_check.append(buy_values_floats[i])
                positive_check_loc.append(buy_locations[i])
        #get location of values detected as zero or negative          
        for i in range(len(buy_values_floats)):
            if type(buy_values_floats[i]) == str:
                continue
            if buy_values_floats[i]<=0:
                zero_or_negative_matrix.append(buy_locations[i])
                print('ZERO OR NEG', zero_or_negative_matrix)
            else:
                asdf = 1 
        for i in range(len(zero_or_negative_matrix)):
            if type(zero_or_negative_matrix[i])== float:
                zero_or_negative = 1
                break
            else:
                zero_or_negative = 'NaN'
                
        if len(positive_check_loc)>1:        
            positive_check_min_loc = positive_check_loc[np.argmin(positive_check)]
            pos_minimum = np.amin(positive_check)
            fx = 0 
            for i in range(len(positive_check)):
                value = positive_check[i]
                coord = positive_check_loc[i]
                for x in range(len(positive_check)):
                    if value> positive_check[x] and coord>positive_check_loc[x]:
                        print('coord higher than min')
                        zero_or_negative = 'Coord problem'
                        break
                    else:
                        continue
                            
        print(buy_values_floats)
        problem = False
        positive_values = []
        distance_ratios = [] 
        for i in range(len(buy_values_floats)):
            
            if zero_or_negative!= 'NaN' or 'Coord problem':
                if buy_values_floats[i] == 'NaN':
                    print('Final values append didnt read number well')
                    final_values.append('Check didnt read number well')
                    problem = True
                    continue
                pos_count1 = 0
                pos_count2= 0
                for x in range(len(zero_or_negative_matrix)): 
                    if buy_values_floats[i]<=0 and buy_locations[i]==zero_or_negative_matrix[x]:
                        final_values.append(buy_values_floats[i])
                        print('final values append zero or neg and zero or neg loc')
                        break
                    elif buy_values_floats[i]>0 and buy_locations[i]==zero_or_negative_matrix[x]:
                        final_values.append(buy_values_floats[i]*-1)
                        print('final val append pos but zero or neg loc')
                        break
                    elif buy_values_floats[i]<=0 and buy_locations[i]!=zero_or_negative_matrix[x]:
                        final_values.append(buy_values_floats[i])    
                        ('final val append neg but not zero or neg loc ')
                        break
                    elif buy_values_floats[i]>0 and buy_locations[i]!=zero_or_negative_matrix[x]:
                        if x==(len(zero_or_negative_matrix)-1):
                            print('positive value not equal to zero or neg')
                            final_values.append(buy_values_floats[i])
                            distance_ratios.append(buy_locations[i]/buy_values_floats[i])
                            break                                                                 
            if zero_or_negative=='Coord problem':
                final_values = [] 
                for i in range(len(buy_locations)):
                    print('final values append coord problem')
                    final_values.append('coord problem')
                    problem = True 
                break 
            elif zero_or_negative == 'NaN':
                if buy_values[1]== 'Five val problem':
                    for i in range(len(buy_locations)):
                        print('final values append five val problem')
                        final_values.append('Five val problem')
                        problem = True 
                        break 
                else:
                    (unique, counts) = np.unique(buy_locations, return_counts=True)
                    if np.max(counts)>1:
                        for i in range(len(buy_locations)):
                            print('final values appending multiple identical value locs')
                            final_values.append('Check multiple identical value locations')
                            problem = True
                        break
                    else:
                        for i in range(len(buy_locations)):
                            final_values.append(buy_values_floats[i])
                            distance_ratios.append(buy_locations[i]/buy_values_floats[i])
                            print('All positive, double check')
                            print('final values appending all positive')
                            problem = True
                        break
                                    
                
        if len(positive_values)>1:               
            print('distance ratios', distance_ratios)
            for i in range(len(positive_check)):  
                print('abs val of Pos loc minus date dist',np.abs(positive_check_loc[i]))
                if np.abs(positive_check_loc[i])<.05 and distance_ratios[i]<-1e-7:
                    final_values = [] 
                    print(' final vals appending distance ratio problem!')
                    problem = True
                    for q in range(len(buy_values_floats)):
                        final_values.append('D.R. problem')
                    break 
        print("Problem", problem)
        if problem == True: 
            img_3_export = filename.split(".")
            exp_image3 = Image.open(filename)
            exp_image3.save('/Users/jpjansen/Desktop/Double Check/'+img_3_export[0]+'problem.png')
    print('BUY VALUES', final_values)
    large_scale_buying.append(final_values)
    if len(final_values)!= 5:
        print('APPENDING PROBLEM HERE')


    

IMG_6722grayimage3.jpeg
height 7596
Height 2 7596
IMG_6722grayimage3.jpeg
['11-187', '4', '0', '21.88-4.01-15.49', '1.09', '0.96', '27.46', '36.81', '14.37', '19.31', '93', '69', '49', '36', '3', '2', '108', '144', '5', '315', '-565', '-1,122', '-1,993', '-97', '03/22', '03/23', '03/24', '03/25', '03/26', '83', '0', '0']
0.8529489204844655
MATCH
buy 1
buy 1 -97
buy 1 location -0.1524486571879936
buy 2 -1993
buy 2 location -0.1524486571879936
buy 3 -1122
buy 3 location -0.1524486571879936
buy 4 -565
buy 4 location -0.1524486571879936
buy 5 315
buy 5 locaion -0.17022116903633489
ZERO OR NEG [-0.1524486571879936]
ZERO OR NEG [-0.1524486571879936, -0.1524486571879936]
ZERO OR NEG [-0.1524486571879936, -0.1524486571879936, -0.1524486571879936]
ZERO OR NEG [-0.1524486571879936, -0.1524486571879936, -0.1524486571879936, -0.1524486571879936]
[-97000.0, -1993000.0, -1122000.0, -565000.0, 315000.0]
final values append zero or neg and zero or neg loc
final values append zero or neg and zero or ne

height 7596
Height 2 7596
IMG_6758grayimage3.jpeg
['1120', '4', '4', '4.2100.02040.48', '4,33', '23.73', '32.23', '9.05', '10.66', '508', '231', '104', '88', '0', '42', '7', '611', '361', '5', '67', '-121', '-77', '-9', '-42', '03/22', '03/23', '03/24', '03/25', '03/26']
0.8517640863612428
MATCH
buy 1
buy 1 -42
buy 1 location -0.12361769352290686
buy 2 -9
buy 2 location -0.12361769352290686
buy 3 -77
buy 3 location -0.12361769352290686
buy 4 -121
buy 4 location -0.12361769352290686
buy 5 67
buy 5 locaion -0.170221169036335
ZERO OR NEG [-0.12361769352290686]
ZERO OR NEG [-0.12361769352290686, -0.12361769352290686]
ZERO OR NEG [-0.12361769352290686, -0.12361769352290686, -0.12361769352290686]
ZERO OR NEG [-0.12361769352290686, -0.12361769352290686, -0.12361769352290686, -0.12361769352290686]
[-42000.0, -9000.0, -77000.0, -121000.0, 67000.0]
final values append zero or neg and zero or neg loc
final values append zero or neg and zero or neg loc
final values append zero or neg and zero or n

BUY VALUES ['Check didnt read number well', 'Check didnt read number well', 'Check didnt read number well', 'Check didnt read number well', 'Check didnt read number well']
IMG_6790grayimage3.jpeg
height 7596
Height 2 7596
IMG_6790grayimage3.jpeg
['11-229', '4', '12.3940.362.99', '29.99', '20.78', '18.91', '24.90', '1,354', '1,124', '938', '953', '245', '0', '2,478', '2,036', '5', '-605', '-14', '0', '-238', '-245', '03/22', '03/23', '03/24', '03/25', '03/26', '4.15040.04040.97', '8', '0', '0']
0.8185887309110058
MATCH
buy 1
buy 1 -245
buy 1 location -0.17022116903633489
buy 2 -238
buy 2 location -0.17022116903633489
buy 3 0
buy 3 location -0.17022116903633489
buy 4 -14
buy 4 location -0.17022116903633489
buy 5 -605
buy 5 locaion -0.17022116903633489
ZERO OR NEG [-0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489, -0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489, -0.170221169

height 7596
Height 2 7596
IMG_6822grayimage3.jpeg
['1128', '13.990.2341.67', '3.33', '16.71', '19.65', '32.514', '37.80', '2,302', '1,369', '1,197', '1.018', '203', '0', '7', '7', '3,702', '2,387', '5', '203', '0', '0', '0', '0', '03/22', '03/23', '03/24', '03/25', '03/26', '83', '0', '0']
0.8529489204844655
MATCH
buy 1
buy 1 0
buy 1 location -0.03949447077409163
buy 2 0
buy 2 location -0.03949447077409163
buy 3 0
buy 3 location -0.03949447077409163
buy 4 0
buy 4 location -0.03949447077409163
buy 5 203
buy 5 locaion -0.17022116903633489
ZERO OR NEG [-0.03949447077409163]
ZERO OR NEG [-0.03949447077409163, -0.03949447077409163]
ZERO OR NEG [-0.03949447077409163, -0.03949447077409163, -0.03949447077409163]
ZERO OR NEG [-0.03949447077409163, -0.03949447077409163, -0.03949447077409163, -0.03949447077409163]
[0.0, 0.0, 0.0, 0.0, 203000.0]
final values append zero or neg and zero or neg loc
final values append zero or neg and zero or neg loc
final values append zero or neg and zero or neg lo

height 7596
Height 2 7596
IMG_6854grayimage3.jpeg
['11-307', '4', '3.970-0.170-4.11', '9,53', '32.07', '34.04', '8.81', '2', '10.29', '324', '350', '377', '193', '1,695', '1,978', '5', '464', '-423', '-213', '7', '-157', '03/22', '03/23', '03/24', '03/25', '03/26', '83', '0', '0']
0.8517640863612428
MATCH
buy 1
buy 1 -157
buy 1 location -0.1018957345971564
buy 2 7
buy 2 location -0.15284360189573465
buy 3 -213
buy 3 location -0.1018957345971564
buy 4 -423
buy 4 location -0.1018957345971564
buy 5 464
buy 5 locaion -0.170221169036335
ZERO OR NEG [-0.1018957345971564]
ZERO OR NEG [-0.1018957345971564, -0.1018957345971564]
ZERO OR NEG [-0.1018957345971564, -0.1018957345971564, -0.1018957345971564]
[-157000.0, 7000.0, -213000.0, -423000.0, 464000.0]
final values append zero or neg and zero or neg loc
positive value not equal to zero or neg
final values append zero or neg and zero or neg loc
final values append zero or neg and zero or neg loc
positive value not equal to zero or neg
Problem F

height 7596
Height 2 7596
IMG_6891grayimage3.jpeg
['2.4900.01040.40', '17.36', '12.96', '55.20', '9', '14.48', '165', '43', '32', '39', '0', '0', '208', '90', '5', '-125', '0', '-25', '0', '0', '03/22', '03/23', '03/24', '03/25', '03/26']
0.8529489204844655
MATCH
buy 1
buy 1 0
buy 1 location -0.17022116903633489
buy 2 0
buy 2 location -0.17022116903633489
buy 3 -25
buy 3 location -0.17022116903633489
buy 4 0
buy 4 location -0.17022116903633489
buy 5 -125
buy 5 locaion -0.17022116903633489
ZERO OR NEG [-0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489, -0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489, -0.17022116903633489, -0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489, -0.17022116903633489, -0.17022116903633489, -0.17022116903633489]
[0.0, 0.0, -25000.0, 0.0, -125000.0]
final values append zero or neg and zero or neg loc
final values append ze

height 7596
Height 2 7596
IMG_6923grayimage3.jpeg
['4.630-0.180-3.74', '4.71', '17.21', '39.694', '14.14', '14.53', '390', '332', '252', '207', '212', '666', '795', '5', '-128', '-200', '-52', '-113', '-183', '03/22', '03/23', '03/24', '03/25', '03/26']
0.8521590310689837
MATCH
buy 1
buy 1 -183
buy 1 location -0.170221169036335
buy 2 -113
buy 2 location -0.170221169036335
buy 3 -52
buy 3 location -0.170221169036335
buy 4 -200
buy 4 location -0.170221169036335
buy 5 -128
buy 5 locaion -0.170221169036335
ZERO OR NEG [-0.170221169036335]
ZERO OR NEG [-0.170221169036335, -0.170221169036335]
ZERO OR NEG [-0.170221169036335, -0.170221169036335, -0.170221169036335]
ZERO OR NEG [-0.170221169036335, -0.170221169036335, -0.170221169036335, -0.170221169036335]
ZERO OR NEG [-0.170221169036335, -0.170221169036335, -0.170221169036335, -0.170221169036335, -0.170221169036335]
[-183000.0, -113000.0, -52000.0, -200000.0, -128000.0]
final values append zero or neg and zero or neg loc
final values append 

height 7596
Height 2 7596
IMG_6951grayimage3.jpeg
['114247', '4.1900.0300.72', '14.37', '3.42', '22.31', '37.61', '542', '321', '322', '207', '0', '49', '528', '912', '5', '49', '0', '0', '-298', '-49', '03/22', '03/23', '03/24', '03/25', '03/26', '83', '0']
0.8529489204844655
MATCH
buy 1
buy 1 -49
buy 1 location -0.15165876777251186
buy 2 -298
buy 2 location -0.15165876777251186
buy 3 0
buy 3 location -0.15165876777251186
buy 4 0
buy 4 location -0.15165876777251186
buy 5 49
buy 5 locaion -0.17022116903633489
ZERO OR NEG [-0.15165876777251186]
ZERO OR NEG [-0.15165876777251186, -0.15165876777251186]
ZERO OR NEG [-0.15165876777251186, -0.15165876777251186, -0.15165876777251186]
ZERO OR NEG [-0.15165876777251186, -0.15165876777251186, -0.15165876777251186, -0.15165876777251186]
[-49000.0, -298000.0, 0.0, 0.0, 49000.0]
final values append zero or neg and zero or neg loc
final values append zero or neg and zero or neg loc
final values append zero or neg and zero or neg loc
final values app

height 7596
Height 2 7596
IMG_6983grayimage3.jpeg
['11447', '0', '5.38-0.08-1.47', '13.3', '24.82', '17.31', '19.67', '741', '788', '624', '550', '336', '137', '1,427', '1,748', '5', '132', '-315', '-/3', '-262', '-199', '03/22', '03/23', '03/24', '03/25', '03/26', '1.8700.0000.00', '8', '0', '0']
0.817798841495524
MATCH
buy 1
buy 1 -199
buy 1 location -0.13151658767772512
buy 2 -262
buy 2 location -0.13151658767772512
buy 3 -/3
buy 3 location -0.13151658767772512
buy 4 -315
buy 4 location -0.13151658767772512
buy 5 132
buy 5 locaion -0.17022116903633489
ZERO OR NEG [-0.13151658767772512]
ZERO OR NEG [-0.13151658767772512, -0.13151658767772512]
ZERO OR NEG [-0.13151658767772512, -0.13151658767772512, -0.13151658767772512]
ZERO OR NEG [-0.13151658767772512, -0.13151658767772512, -0.13151658767772512, -0.13151658767772512]
[-199000.0, -262000.0, -73000.0, -315000.0, 132000.0]
final values append zero or neg and zero or neg loc
final values append zero or neg and zero or neg loc
final val

In [10]:
workbook = xlsxwriter.Workbook('3_28_21_Test_Data.xlsx',options={'nan_inf_to_errors': True})
cell_format = workbook.add_format()
ten_day_ma = np.asarray(image_one_data)
inflow_data = np.asarray(inflows)
large_scale_buying_exp = np.asarray(large_scale_buying)
free_float_exp = np.asarray(free_float)
wks = workbook.add_worksheet()


wks.write(0,0,'10 day name')
wks.write(0,1,'10 day MA (USD)')
wks.write(0,2,'inflow/outflow (USD) name')
wks.write(0,3,'inflow')
wks.write(0,4,'outflow')
wks.write(0,5,'large scale buying (USD) name')
wks.write(0,6,'first buying')
wks.write(0,7,'second buying')
wks.write(0,8,'third buying')
wks.write(0,9,'fourth buying')
wks.write(0,10,'fifth buying')
wks.write(0,11,'float name')
wks.write(0,12,'free float in millions')
for i in range(len(inflows)):
    wks.write(i+1,0,ten_day_ma[i,0])
    wks.write(i+1,1,float(ten_day_ma[i,1]))
    wks.write(i+1,2,inflow_data[i,0])
    wks.write(i+1,3,float(inflow_data[i,1]))
    wks.write(i+1,4,float(outflows[i]))
    wks.write(i+1,5,image_threes[i,0])
    for x in range(5):
        arr = np.asarray(large_scale_buying_exp[i])
        try:
            floater = float(arr[x])
            print(arr[x])
            cell_format.set_bg_color('white')
            wks.write(i+1,x+6,floater)
        except ValueError:
            print(arr[x])
            cell_format.set_bg_color('red')
            wks.write(i+1,x+6,arr[x])
        except IndexError:
            wks.write(i+1,x+6, 'Index Error')
    cell_format.set_bg_color('white')
    wks.write(i+1,11,free_float_exp[i,0])
    wks.write(i+1,12,float(free_float_exp[i,1]))
workbook.close()

In [ ]:
from PIL import Image
for filename in os.listdir(os.getcwd()):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        if filename == '.DS_Store':
            continue
        #filename = filename.split(".")
        img = Image.open(filename).convert('LA')
        img = img.resize((7350,16000),resample=1 ) 
        export = filename.split(".")
        
        img.save(export[0]+'big2.png')

    

In [48]:
print(
        

[-5000.0, -301000.0, 0.0, 881000.0, 1059000.0]


In [ ]:
for filename in os.listdir(os.getcwd()):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        if filename == '.DS_Store':
            continue
        print(filename)
        #,cv.IMREAD_GRAYSCALE
        img = cv.imread(filename)
        ret,thresh1 = cv.threshold(img,245,255,cv.THRESH_BINARY)
        kernel = np.ones((5,5), np.uint8)
        img_dilation = cv.dilate(thresh1, kernel, iterations=10) 
        cv.imshow('Dilation', thresh1) 
         #--psm 12 -c tessedit_char_whitelist= 0123456789
        text = pytesseract.image_to_string(thresh1,config='--psm 6 --oem 3 -c tessedit_char_whitelist=/.,-0123456789')
        nltk_tokens = nltk.word_tokenize(text)
        for i in range(len(nltk_tokens)):
            #print(i, nltk_tokens[i])
            if re.match(r"[0-9][0-9]/[0-9][0-9]",nltk_tokens[i])!=None:
                print('Buy1',nltk_tokens[i-1],'Buy2',nltk_tokens[i-2],'Buy3',nltk_tokens[i-3],'Buy4',nltk_tokens[i-4],'Buy5',nltk_tokens[i-5])
                break
        cv.waitKey(0) 
        
    
    

In [ ]:
string = '13,45,23'
if "." not in string:
    commas = []
    for i in range(len(string)):
        
        if string[i]==",":
            commas.append(i)
    replace = np.max(commas)
    new_str = list(string)
    del(new_str[replace])
    new_str.insert(replace,".")
    print(new_str)
    final = ''.join(new_str)
    final.replace(",","")
    print(final)

In [32]:
print(positive_values)
print(positive_check_loc)




[-440000.0, -1164000.0, 334000.0, 1092000.0, 1459000.0, -440000.0, -1164000.0, 334000.0, 1092000.0, 1459000.0, 334000.0, -440000.0, -1164000.0, 334000.0, 1092000.0, 1459000.0, 1092000.0, -440000.0, -1164000.0, 334000.0, 1092000.0, 1459000.0, 1459000.0, -440000.0, -1164000.0, 334000.0, 1092000.0, 1459000.0]
[-0.11413902053712477, -0.15165876777251186, -0.17022116903633489]


In [34]:
for i in range(len(large_scale_buying_exp)):
    row = large_scale_buying_exp[i,:]
    #print(row)
    for q in range(len(row)):
        try:
            floater = float(row[q])
            print(type(floater))
            print(floater)
        except ValueError:
            print(row[q])
        #if type(row[q])== str:
         #   print('string')
        #if type(row[q])==float:
            #print('float')

<class 'float'>
-1216000.0
<class 'float'>
-6288000.0
<class 'float'>
1770000.0
<class 'float'>
5225000.0
<class 'float'>
8500000.0
<class 'float'>
-2850000.0
Check think its negative but not sure
<class 'float'>
-124000.0
Check think its negative but not sure
<class 'float'>
448000.0
<class 'float'>
49000.0
<class 'float'>
20000.0
<class 'float'>
597000.0
<class 'float'>
1051000.0
<class 'float'>
1319000.0
<class 'float'>
-150000.0
Check didnt read number well
<class 'float'>
-726000.0
<class 'float'>
1187000.0
<class 'float'>
1363000.0
<class 'float'>
-8000000.0
<class 'float'>
4000000.0
<class 'float'>
9000000.0
<class 'float'>
16000000.0
<class 'float'>
24000000.0
<class 'float'>
-218000.0
<class 'float'>
-9616000.0
Check think its negative but not sure
<class 'float'>
3764000.0
<class 'float'>
8354000.0
<class 'float'>
0.0
<class 'float'>
0.0
<class 'float'>
7000.0
Check think its negative but not sure
<class 'float'>
233000.0
<class 'float'>
0.0
<class 'float'>
0.0
<class 'float'

SyntaxError: invalid syntax (<ipython-input-28-7974c784ec40>, line 131)

[1 1 3 1 2 1 1]
3


In [5]:
print(final_multiplier)

1000.0


In [47]:
os.chdir('/Users/jpjansen/Desktop/Double Check')

large_scale_buying = [] 
counter = 0 
image_threes = np.asarray(image_threes)
image_three_resave = []
for filename in sorted(os.listdir(os.getcwd())):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        if filename == '.DS_Store':
            continue
        print(filename)
        img = cv.imread(filename)
        height = img.shape[0]
        print('height', height)
        ret,thresh1 = cv.threshold(img,240,255,cv.THRESH_BINARY)
        height_2 = thresh1.shape[0]
        print('Height 2', height_2)
        kernel = np.ones((2,2), np.uint8)
        img_dilation = cv.erode(thresh1, kernel, iterations=3) 
        boxes = pytesseract.image_to_data(thresh1,output_type= Output.DICT,config='--psm 6 --oem 3 -c tessedit_char_whitelist=/.,-0123456789')
        good_words = [] 
        good_top = [] 
        remove = ['/',',','.','','-', None]
        for z in range(len(boxes['text'])):
            #print(len(boxes['text'][z]))
            if len(boxes['text'][z])==0:
                #print('Zero')
                continue
            if boxes['text'][z] not in remove:
                good_words.append(boxes['text'][z])
                good_top.append(boxes['top'][z])
        print('good words', good_words)
        print('good top', good_top)

IMG_4670grayimage3problem.png
height 7596
Height 2 7596
good words ['1017', '4', '2.220-0.060-2.63', '6.42', '7.52', '24.47', '31.49', '15.01', '15.09', '4,288', '3,332', '2,044', '2,050', '875', '1,025', '6,251', '7,362', '5', '1,363', '1,187', '-726', '-2/0', '-150', '01/25', '01/26', '01/27', '01/28', '01/29', '83', '0']
good top [107, 293, 674, 1229, 1232, 1637, 1823, 2414, 2570, 2961, 3138, 3351, 3376, 3591, 3564, 4100, 4100, 4791, 5186, 5270, 5834, 5834, 5834, 6479, 6479, 6479, 6479, 6479, 6924, 7023]
IMG_4682grayimage3.jpeg
height 7596
Height 2 7596
good words ['1029', '4', '5.4240.0140.18', '19.59', '28.07', '26.47', '25.87', '748', '793', '730', '553', '0', '0', '1,301', '1,523', '5', '305', '233', '-420', '7', '0', '0', '01/25', '01/26', '01/27', '01/28', '01/29', '83']
good top [107, 293, 674, 1241, 1358, 2126, 2375, 2928, 2886, 2947, 3117, 3633, 3633, 3998, 3998, 4689, 5084, 5183, 5500, 5216, 5504, 5504, 6377, 6377, 6377, 6377, 6377, 6924]
IMG_4742grayimage3problem.png
heig

In [24]:

print(asdf)

[('VISL', 'IMG_4966gray.png') ('VISL', 'IMG_4966gray.png', 1000)
 ('CTRM', 'IMG_4970gray.png') ('CTRM', 'IMG_4970gray.png', 1000)
 ('SNDL', 'IMG_4974gray.png') ('SNDL', 'IMG_4974gray.png', 1000)
 ('OCGN', 'IMG_4978gray.png') ('OCGN', 'IMG_4978gray.png', 1000)
 ('FLNT', 'IMG_4982gray.png') ('FLNT', 'IMG_4982gray.png', 1000)
 ('XSPA', 'IMG_4986gray.png') ('XSPA', 'IMG_4986gray.png', 1000)
 ('ACST', 'IMG_4990gray.png') ('ACST', 'IMG_4990gray.png', 1000)
 ('NAKD', 'IMG_4994gray.png') ('NAKD', 'IMG_4994gray.png', 1000)
 ('CCIV', 'IMG_4998gray.png') ('CCIV', 'IMG_4998gray.png', 1000)
 ('NSPR', 'IMG_5003gray.png') ('NSPR', 'IMG_5003gray.png', 1000)
 ('NRBO', 'IMG_5007gray.png') ('NRBO', 'IMG_5007gray.png', 1000)
 ('PPBT', 'IMG_5011gray.png') ('PPBT', 'IMG_5011gray.png', 1000)
 ('TGC', 'IMG_5015gray.png') ('TGC', 'IMG_5015gray.png', 1000)
 ('LAIX', 'IMG_5019gray.png') ('LAIX', 'IMG_5019gray.png', 1000)
 ('UK', 'IMG_5023gray.png') ('UK', 'IMG_5023gray.png', 1000)
 ('AGFS', 'IMG_5027gray.png') (

IMG_5167gray.png


[-1.7408648676434628e-07, -1.6073764781523598e-07]
-1.6741206728979112e-07
